In [1]:
import torch

from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertForMaskedLM

torch_device=torch.device('cuda')

bert_model_mlm = BertForMaskedLM.from_pretrained('bert-base-uncased')
bert_model_mlm.eval()
bert_model_mlm.to(torch_device)

for param in bert_model_mlm.parameters():
    param.requires_grad = False

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

bert_id2tok = dict()
for tok, tok_id in bert_tokenizer.vocab.items():
    bert_id2tok[tok_id] = tok

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
import pickle
import numpy as np

ft_compiled_path = "../data/jigsaw/ft_compiled.npy" # Embeddings generated from the vocabulary
data_vocab_path = "../data/jigsaw/data_vocab.bin"

MAX_BERT_LEN=256
MAX_COSINE_DIST=0.3
BERT_VOCAB_QTY=30000

num_threads=8
K=10

In [3]:
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter

_spacy_tok = SpacyWordSplitter(language='en_core_web_sm', pos_tags=False).split_words

def tokenizer(s: str):
    s = s.lower()
    return [w.text for w in _spacy_tok(s)]

In [31]:
"delete" in bert_tokenizer.vocab

False

In [5]:
bert_id2tok[2]

'[unused1]'

In [6]:
bert_tokenizer.vocab['d']

3231

In [7]:
# Returns arrays of arrays if there's an OOV word or an empty array instead
# Each array element is a tuple: 
# position of OOV word (with respect to the original tokenizer), sent for BERT tokenizer
def get_bert_masked_inputs(toks, bert_tokenizer):
    res = []
    
    oov_pos = []
    bert_vocab = bert_tokenizer.vocab
    
    for i in range(len(toks)):
        if toks[i] not in bert_vocab:
            oov_pos.append(i)
            

    for pos in oov_pos:
        res.append( (pos, '[CLS] %s [MASK] %s [SEP]' % 
                     (' '.join(toks[0:pos]), ' '.join(toks[pos+1:])) ) )
        
    return res

In [8]:
get_bert_masked_inputs(tokenizer('This is a *strangge* sentence.'), bert_tokenizer)

[(4, '[CLS] this is a * [MASK] * sentence . [SEP]')]

In [9]:
toks = bert_tokenizer.tokenize('[CLS] what the [MASK] are you doing here ? [SEP]')
toks

['[CLS]', 'what', 'the', '[MASK]', 'are', 'you', 'doing', 'here', '?', '[SEP]']

In [10]:
tokenizer("don't  couldn't can't you're I'm")

['do', "n't", 'could', "n't", 'ca', "n't", 'you', "'re", 'i', "'m"]

In [11]:
bert_tokenizer.tokenize("don't  couldn't can't you're I'm")

['don',
 "'",
 't',
 'couldn',
 "'",
 't',
 'can',
 "'",
 't',
 'you',
 "'",
 're',
 'i',
 "'",
 'm']

In [34]:
bert_tokenizer.tokenize("insert")

['insert']

In [12]:
tokenizer("You ' re right. It ' s a miracle! You'd been deceived!") # 've', 're', 's', 'd', 'll'

['you',
 "'",
 're',
 'right',
 '.',
 'it',
 "'",
 's',
 'a',
 'miracle',
 '!',
 'you',
 "'d",
 'been',
 'deceived',
 '!']

In [13]:
from collections import namedtuple
# pos_oov is OOV index with respect to the original (not BERT) tokenizer!!!
UtterData = namedtuple('SentData', ['batch_sent_id', 'pos_oov', 'tok_ids', 'oov_token'])

def get_batch_data(torch_device, tokenizer, bert_tokenizer, sent_list, max_len=MAX_BERT_LEN):
    
    batch_data_raw = []
    batch_max_seq_qty = 0
    batch_sent_id = -1
    for sent in sent_list:
        batch_sent_id += 1
        sent_toks = tokenizer(sent)
        for sent_oov_pos, text in get_bert_masked_inputs(sent_toks, bert_tokenizer):
            # To accurately get what is the position of [MASK] according
            # to BERT tokenizer, we need to re-tokenize the sentence using
            # the BERT tokenizer
            all_bert_toks = bert_tokenizer.tokenize(text)
            bert_toks = all_bert_toks[0:max_len] # 512 is the max. Bert seq. length

            tok_ids = bert_tokenizer.convert_tokens_to_ids(bert_toks)
            pos_oov = None
            for i in range(len(bert_toks)):
                if bert_toks[i] == '[MASK]':
                    pos_oov = i
                    break
            assert(pos_oov is not None or len(all_bert_toks) > max_len)
            if pos_oov is not None:
                tok_qty = len(tok_ids)
                batch_max_seq_qty = max(batch_max_seq_qty, tok_qty)
                batch_data_raw.append( 
                    UtterData(batch_sent_id=batch_sent_id, 
                              pos_oov=sent_oov_pos, 
                              tok_ids=tok_ids, 
                              oov_token=sent_toks[sent_oov_pos]))
            
    batch_qty = len(batch_data_raw)
    tok_ids_batch = np.zeros( (batch_qty, batch_max_seq_qty), dtype=np.int64) # zero is a padding symbol
    for k in range(batch_qty):
        tok_ids = batch_data_raw[k].tok_ids
        tok_ids_batch[k, 0:len(tok_ids)] = tok_ids
        
                   
    tok_ids_batch = torch.from_numpy(tok_ids_batch).to(device=torch_device) 
    
    return batch_data_raw, tok_ids_batch

In [14]:
import torch
import numpy as np
from collections import namedtuple
import sys

BertPredProbs = namedtuple('BertPred', ['batch_sent_id', 'pos_oov', 'logits'])

def get_bert_preds_for_words_batch(torch_device, bert_model_mlm, 
                                   batch_data_raw, tok_ids_batch, # comes from get_batch_data
                                   word_ids, # a list of IDS for which we generate logits
                                   max_len=MAX_BERT_LEN):

    seg_ids = torch.zeros_like(tok_ids_batch, device=torch_device)
    
    batch_qty = len(batch_data_raw)
    
    # Main BERT model see modeling.py in https://github.com/huggingface/pytorch-pretrained-BERT
    bert = bert_model_mlm.bert 
    # cls is an instance of BertOnlyMLMHead (see https://github.com/huggingface/pytorch-pretrained-BERT)
    cls = bert_model_mlm.cls
    # predictions are of the type BertLMPredictionHead (see https://github.com/huggingface/pytorch-pretrained-BERT)
    predictions = cls.predictions
    transform = predictions.transform
   
    # We don't use the complete decoding matrix, but only selected rows
    word_ids = torch.from_numpy(np.array(word_ids, dtype=np.int64)).to(device=torch_device)
                                
    weight = predictions.decoder.weight[word_ids,:]
    bias = predictions.bias[word_ids]

    # Transformations from the main BERT model
    sequence_output, _= bert(tok_ids_batch, seg_ids, attention_mask=None, output_all_encoded_layers=False)
    # Transformations from the BertLMPredictionHead model with the restricted last layer
    hidden_states = transform(sequence_output)    
    logits = torch.nn.functional.linear(hidden_states, weight) + bias                            
                                        
    logits=logits.detach().cpu().numpy()
    
    res = []
    
    for k in range(batch_qty):
        
        pos_oov = batch_data_raw[k].pos_oov         
        res.append( BertPredProbs(batch_sent_id = batch_data_raw[k].batch_sent_id,
                             pos_oov = pos_oov,
                             logits = logits[k, pos_oov]
                            ) 
                  )
        
    return res

In [15]:
bert_tokenizer.convert_tokens_to_ids(['hell', 'fuck', 'heck', 'devil', 'shit', 'doing', 
                                      'doin', 'wearing', 'making', 'thinking', 'all', 'test'])

[3109, 6616, 17752, 6548, 4485, 2725, 24341, 4147, 2437, 3241, 2035, 3231]

In [16]:
sent_list = ['What the fcuk are you doingg here?',
             'This is a *strangge* sentence']

batch_data_raw, tok_ids_batch = get_batch_data(torch_device, 
                                                tokenizer, 
                                                bert_tokenizer, 
                                                sent_list,
                                                MAX_BERT_LEN)

get_bert_preds_for_words_batch(torch_device,
                               bert_model_mlm, 
                               batch_data_raw, tok_ids_batch,
                               bert_tokenizer.convert_tokens_to_ids(['hell', 'fuck', 'heck', 'devil', 'shit', 'doing', 
                                                                  'doin', 'wearing', 'making', 'thinking']))

[BertPred(batch_sent_id=0, pos_oov=2, logits=array([ 6.0060143,  3.6601357, -0.717967 , -2.1604285,  1.0156769,
        -2.321704 , -7.334852 , -6.295347 , -3.6437619, -7.7260413],
       dtype=float32)),
 BertPred(batch_sent_id=0, pos_oov=5, logits=array([ 0.8385326, -1.348005 , -2.5296175, -4.8170433, -2.2217095,
         4.484024 , -2.3674998, -4.7202344, -3.995442 , -4.7819214],
       dtype=float32)),
 BertPred(batch_sent_id=1, pos_oov=4, logits=array([-3.751286 , -3.4677486, -4.074821 , -5.3647747, -2.4487138,
        -1.9118499, -4.7675657, -6.888247 , -1.6076685, -3.1759496],
       dtype=float32))]

In [17]:
#bert_model_mlm.to('cpu')
#torch.zeros(3,device=torch.device("cuda"))

In [18]:
bert_tokenizer.tokenize('б')

['б']

In [19]:
from typing import *
from overrides import overrides
import allennlp
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.dataset_readers import DatasetReader

from allennlp.data import Instance
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token

from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter
from allennlp.data.token_indexers import WordpieceIndexer, SingleIdTokenIndexer
from allennlp.data.fields import TextField, SequenceLabelField, LabelField, MetadataField, ArrayField
class MemoryOptimizedTextField(TextField):
    @overrides
    def __init__(self, tokens: List[str], token_indexers: Dict[str, TokenIndexer]) -> None:
        self.tokens = tokens
        self._token_indexers = token_indexers
        self._indexed_tokens: Optional[Dict[str, TokenList]] = None
        self._indexer_name_to_indexed_token: Optional[Dict[str, List[str]]] = None
        # skip checks for tokens
    @overrides
    def index(self, vocab):
        super().index(vocab)
        self.tokens = None # empty tokens


In [20]:
import pickle
import numpy as np

ft_compiled_path = "../data/jigsaw/ft_compiled.npy" # Embeddings generated from the vocabulary
data_vocab_path = "../data/jigsaw/data_vocab.bin"

In [21]:
fasttext_embeds = np.load(ft_compiled_path)
vocab=pickle.load(open(data_vocab_path,'rb'))

In [22]:
# These are *MAIN* vocabulary word IDs for words in the BERT vocabulary.
bert_vocab_term_glob_ids = []
bert_vocab_term_bert_ids = []

for tok, bert_tok_id in bert_tokenizer.vocab.items():
    glob_tok_id = vocab.get_token_index(tok)
    if glob_tok_id > 1:
        bert_vocab_term_glob_ids.append(glob_tok_id)
        bert_vocab_term_bert_ids.append(bert_tok_id)
        
bert_vocab_term_glob_ids = np.array(bert_vocab_term_glob_ids)
bert_vocab_term_bert_ids = np.array(bert_vocab_term_bert_ids)
fasttext_embeds[bert_vocab_term_glob_ids].shape

(22778, 300)

In [23]:
max(bert_vocab_term_bert_ids)

29611

In [24]:
import nmslib, time

M = 30
efC = 200

num_threads = 0
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC, 'post' : 0}
print('Index-time parameters', index_time_params)

# Space name should correspond to the space name 
# used for brute-force search
space_name='cosinesimil'


# Intitialize the library, specify the space, the type of the vector and add data points 
index = nmslib.init(method='hnsw', space=space_name, data_type=nmslib.DataType.DENSE_VECTOR) 
index.addDataPointBatch(fasttext_embeds[bert_vocab_term_glob_ids], bert_vocab_term_bert_ids)

# Create an index
start = time.time()
index_time_params = {'M': M, 'indexThreadQty': num_threads, 'efConstruction': efC}
index.createIndex(index_time_params) 
end = time.time() 
print('Index-time parameters', index_time_params)
print('Indexing time = %f' % (end-start))


Index-time parameters {'M': 30, 'indexThreadQty': 0, 'efConstruction': 200, 'post': 0}
Index-time parameters {'M': 30, 'indexThreadQty': 0, 'efConstruction': 200}
Indexing time = 4.952125


In [25]:
# Setting query-time parameters
efS = 200
K=10
query_time_params = {'efSearch': efS}
print('Setting query-time parameters', query_time_params)
index.setQueryTimeParams(query_time_params)

Setting query-time parameters {'efSearch': 200}


In [26]:
import pickle
with open("../data/jigsaw/val_ds.bin", "rb") as f:
    val_ds = pickle.load(f)

In [27]:
from itertools import groupby
import re

def remove_extra_chars(s, max_qty=2):
    res = [c * min(max_qty, len(list(group_iter))) for c, group_iter in groupby(s)] 
    return ''.join(res)

def is_apost_token(s):
    return re.match(r"'[a-z]{1,3}$", s) is not None


def replace_by_patterns(tokenizer, s, replace_dict):
    res = tokenizer(s)
    for pos, repl in replace_dict.items():
        res[pos] = repl
    return ' '.join(res)
    

In [ ]:
import time, gc
from scipy.special import softmax


DEBUG_PRINT=False

t0 = time.time()
preds = []
all_sents = [' '.join(t['tokens']) for t in val_ds]
    #print(s)
    #preds.append(get_bert_top_preds(tokenizer, bert_tokenizer, s, 2))
    #preds.append(get_bert_masked_inputs(tokenizer(s), bert_tokenizer, sent))


batch_qty_step = 20

for batch_start_sent_id in range(0, len(all_sents), batch_qty_step):
    print('Batch start', batch_start_sent_id)
    
    batch_qty = min(batch_qty_step, len(all_sents) - batch_start_sent_id)

    batch_sents = [all_sents[k] for k in range(batch_start_sent_id,
                                               batch_start_sent_id + batch_qty)]

    replace_dict = {k : dict() for k in range(0,batch_qty)} 
    
    # batch_data raw contains elements
    # UtterData = namedtuple('SentData', ['batch_sent_id', 'pos_oov', 'tok_ids', 'oov_token')
    # NOTE: pos_oov is OOV index with respect to the original (not BERT) tokenizer!!!
    #
    # tok_ids_batch is a Tensor with padded Bert-specific token IDs ready
    # to be fed into a BERT model
    batch_data_raw, tok_ids_batch = get_batch_data(torch_device,
                                                 tokenizer, bert_tokenizer,
                                                 batch_sents, 
                                                 MAX_BERT_LEN)
    
    query_arr = []
    query_tok_oov_id = []
    
    for e in batch_data_raw: 
        w = e.oov_token
        wCompr = remove_extra_chars(w)
        wid = vocab.get_token_index(wCompr)
        if w != wCompr:
            if wid < 2:
                wid = vocab.get_token_index(w)
         
        query_arr.append(fasttext_embeds[wid])
        query_tok_oov_id.append(wid)
        
    query_arr = np.array(query_arr)
    query_matrix = np.array(query_arr)
    query_qty = query_matrix.shape[0]
    
    if DEBUG_PRINT: print('Query matrix shape:', query_matrix.shape)
    
    start = time.time() 
    # nbrs is array of tuples (neighbor array, distance array)
    # For cosine, the distance is 1 - cosine similarity
    # k-NN search returns Bert-specific token IDs
    nbrs = index.knnQueryBatch(query_matrix, k = K, num_threads = num_threads)
    end = time.time() 
    if DEBUG_PRINT: 
        print('kNN time total=%f (sec), per query=%f (sec), per query adjusted for thread number=%f (sec)' % 
          (end-start, float(end-start)/query_qty, num_threads*float(end-start)/query_qty))
    
    neighb_tok_ids=set()
    
    for qid in range(query_qty):
        if query_tok_oov_id[qid] > 1:
            nbrs_ids = nbrs[qid][0]
            nbrs_dist = nbrs[qid][1]
            
            nqty = len(nbrs_ids)
            for t in range(nqty):
                if nbrs_dist[t] < MAX_COSINE_DIST:
                    assert(nbrs_ids[t] < BERT_VOCAB_QTY)
                    neighb_tok_ids.add(nbrs_ids[t])
                 
    
    neighb_tok_ids = list(neighb_tok_ids)
    
    preds = get_bert_preds_for_words_batch(torch_device,
                                           bert_model_mlm,
                                           batch_data_raw, tok_ids_batch,
                                           neighb_tok_ids)
    
    assert(len(preds) == query_qty)
    for qid in range(query_qty):
        e = batch_data_raw[qid]
        glob_sent_id = batch_start_sent_id + e.batch_sent_id
        assert(batch_sents[e.batch_sent_id] == all_sents[glob_sent_id])
        if is_apost_token(e.oov_token) or e.oov_token == "n't":
            # Thing's like "I don't" or "You're" are tokenized as do "I do n't" or "You 're'"
            pass # TODO fix this
        elif query_tok_oov_id[qid] > 1:  
            # Let's map neighbor IDs from each queries to respective 
            # logits from the prediction set
            logit_map = dict() # from Bert-specific token IDs to predicted logits
            assert(len(preds[qid].logits) == len(neighb_tok_ids))
            for i in range(len(neighb_tok_ids)):
                logit_map[neighb_tok_ids[i]] = preds[qid].logits[i]

            e = batch_data_raw[qid]
            if DEBUG_PRINT: 
                print(all_sents[glob_sent_id])
                print("### OOV ###", e.oov_token)
                print([bert_id2tok[bert_tok_id] for bert_tok_id in nbrs[qid][0]])

            nbrs_sel_logits = []
            nbrs_sel_toks = []
            nbrs_sel_dists = []
            
            nbrs_ids = nbrs[qid][0]
            nbrs_dist = nbrs[qid][1]
            
            #print('Logit map:', logit_map)
            #print('neighb_tok_ids', neighb_tok_ids)
            
            nqty = len(nbrs_ids)
            for t in range(nqty):
                bert_tok_id = nbrs_ids[t]
                # nid is Bert-speicifc token ID
                if not bert_tok_id in neighb_tok_ids:
                    if DEBUG_PRINT: 
                        print('Missing %s distance %g ' 
                              % (bert_id2tok[bert_tok_id],
                                 nbrs_dist[t]))
                else:
                    if nbrs_dist[t] < MAX_COSINE_DIST:
                        nbrs_sel_logits.append(logit_map[bert_tok_id])
                        nbrs_sel_toks.append(bert_id2tok[bert_tok_id]) 
                        nbrs_sel_dists.append(nbrs_dist[t])
            
            if nbrs_sel_logits:
                nbrs_softmax = softmax(np.array(nbrs_sel_logits))
                nbrs_simil = 1 - np.array(nbrs_sel_dists)
                nbrs_simil_adj = nbrs_softmax * nbrs_simil 
                
                best_tok_id = np.argmax(nbrs_simil_adj)
                
                #print("batch sent id:",e.batch_sent_id, e.pos_oov, best_tok_id)
                #print(replace_dict[e.batch_sent_id])
                assert(not e.pos_oov in replace_dict[e.batch_sent_id])
                replace_dict[e.batch_sent_id][e.pos_oov] = nbrs_sel_toks[best_tok_id]
                
                if DEBUG_PRINT: 
                    print('Selected info, best_tok:', nbrs_sel_toks[best_tok_id])
                    for k in range(len(nbrs_sel_logits)):
                        print(nbrs_sel_toks[k], nbrs_softmax[k], 
                              nbrs_sel_dists[k], nbrs_simil_adj[k])
            else:
                if DEBUG_PRINT: print('Nothing found!')

            #if DEBUG_PRINT: print(preds[qid])
            if DEBUG_PRINT: 
                print("====================================================================")

    
    
    #gc.collect()
    #torch.cuda.empty_cache()
    for k in range(0, batch_qty):
        src_sent = batch_sents[k]
        rd = replace_dict[k]
        #print('Replacement dict:', rd)
        dst_sent = replace_by_patterns(tokenizer, src_sent, rd)
        if DEBUG_PRINT:
            print("====================================================================")
            print('Replacement dict:', rd)
            print(src_sent)
            print('------------')
            print(dst_sent)
            print("====================================================================")
    
    #break
    
t1 = time.time()
print('# of sentences:', len(all_sents), ' time elapsed:', t1 - t0)

Batch start 0
Batch start 20
Batch start 40
Batch start 60
Batch start 80
Batch start 100
Batch start 120
Batch start 140
Batch start 160
Batch start 180
Batch start 200
Batch start 220
Batch start 240
Batch start 260
Batch start 280
Batch start 300
Batch start 320
Batch start 340
Batch start 360
Batch start 380
Batch start 400
Batch start 420
Batch start 440
Batch start 460
Batch start 480
Batch start 500
Batch start 520
Batch start 540
Batch start 560
Batch start 580
Batch start 600
Batch start 620
Batch start 640
Batch start 660
Batch start 680
Batch start 700
Batch start 720
Batch start 740
Batch start 760
Batch start 780
Batch start 800
Batch start 820
Batch start 840
Batch start 860
Batch start 880
Batch start 900
Batch start 920
Batch start 940
Batch start 960
Batch start 980
Batch start 1000
Batch start 1020
Batch start 1040
Batch start 1060
Batch start 1080
Batch start 1100
Batch start 1120
Batch start 1140
Batch start 1160
Batch start 1180
Batch start 1200
Batch start 1220
Ba

In [29]:
#torch_device=torch.device('cuda')
#bert_model_mlm.to(torch_device)

In [30]:
(350/7979)*200000/60/60

2.436952556015095